In [0]:
from pyspark.sql.functions import (
    col, count, date_trunc, when
)

In [0]:
alerts_by_hour = (
    spark.table("healthcare_analytics.gold_patient_alerts")
    .withColumn("alert_hour", date_trunc("hour", col("alert_generated_ts")))
    .groupBy("alert_hour")
    .agg(count("*").alias("total_alerts"))
    .orderBy("alert_hour")
)

alerts_by_hour.display()

alert_hour,total_alerts
2026-01-18T22:00:00.000Z,424


In [0]:
alerts_by_severity = (
    spark.table("healthcare_analytics.gold_patient_alerts")
    .groupBy("overall_severity")
    .agg(count("*").alias("alert_count"))
    .orderBy(col("alert_count").desc())
)

alerts_by_severity.display()

overall_severity,alert_count
CRITICAL,224
HIGH,111
NORMAL,89


In [0]:
patients_by_risk = (
    spark.table("healthcare_analytics.gold_patient_risk_snapshot")
    .groupBy("risk_group")
    .agg(count("*").alias("patient_count"))
    .orderBy(col("patient_count").desc())
)

patients_by_risk.display()

risk_group,patient_count
STABLE,313
CRITICAL,111


In [0]:

alert_type_distribution = (
    spark.table("healthcare_analytics.gold_patient_alerts")
    .selectExpr(
        "heart_rate_alert",
        "oxygen_alert",
        "bp_alert"
    )
    .selectExpr(
        """
        stack(
            3,
            'heart_rate', heart_rate_alert,
            'oxygen', oxygen_alert,
            'bp', bp_alert
        ) as (alert_type, alert_status)
        """
    )
    .where(col("alert_status") != "NORMAL")
    .groupBy("alert_type")
    .agg(count("*").alias("alert_count"))
    .orderBy(col("alert_count").desc())
)

alert_type_distribution.display()

alert_type,alert_count
oxygen,271
bp,184
heart_rate,176


In [0]:
high_risk_patients = (
    spark.table("healthcare_analytics.gold_patient_risk_snapshot")
    .where(col("risk_group") == "CRITICAL")
    .select(
        "patient_id",
        "heart_rate",
        "oxygen_level",
        "systolic_bp",
        "overall_severity",
        "snapshot_ts"
    )
)

high_risk_patients.display()

patient_id,heart_rate,oxygen_level,systolic_bp,overall_severity,snapshot_ts
243,113,90,103,HIGH,2026-01-19T05:07:17.869Z
481,108,99,105,HIGH,2026-01-19T05:07:17.869Z
296,101,98,159,HIGH,2026-01-19T05:07:17.869Z
78,66,92,141,HIGH,2026-01-19T05:07:17.869Z
322,95,91,142,HIGH,2026-01-19T05:07:17.869Z
375,59,95,156,HIGH,2026-01-19T05:07:17.869Z
155,66,99,155,HIGH,2026-01-19T05:07:17.869Z
193,118,97,101,HIGH,2026-01-19T05:07:17.869Z
81,117,98,127,HIGH,2026-01-19T05:07:17.869Z
436,77,90,149,HIGH,2026-01-19T05:07:17.869Z
